<a href="https://colab.research.google.com/github/Nedu21/Pytorch-deep-learning-projects-/blob/main/Defect_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from pathlib import Path

import numpy as np
import random
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.modules.batchnorm import BatchNorm2d
from torchvision import transforms
import torch.nn as nn
from PIL import Image

In [ ]:
# Ensuring repeatablility across runs in PyTorch for consistent results.

def set_seed(seed: int = 42) -> None:
    """Seed Python, NumPy, and PyTorch (CPU & all GPUs) and
    make cuDNN run in deterministic mode to ensure reproducibility."""

    # ---- Seed Python's built-in random module -----------------------
    random.seed(seed);
    # ---- Seed NumPy's random number generator -----------------------
    np.random.seed(seed);

    # ---- Seed PyTorch (CPU & all GPUs) ------------------------------
    torch.manual_seed(seed);            # Seed for CPU operations
    torch.cuda.manual_seed_all(seed);   # Seed for all GPU operations

    # ---- cuDNN: Configure for repeatable convolutions ---------------
    # This ensures that cudnn algorithms are deterministic.
    torch.backends.cudnn.deterministic = True;
    # Disable cuDNN benchmarking to ensure consistent execution speed (can be slower).
    torch.backends.cudnn.benchmark     = False;

# Global seed for all random operations.
SEED = 42;
# Apply the global seed to all relevant libraries.
set_seed(SEED);
print(f"Global seed set to {SEED} — main process is now deterministic.");

# Define worker_init_fn function for DataLoader workers.
def worker_init_fn(worker_id: int) -> None:
    """Re-seed each DataLoader worker so their RNGs don't collide.
    This ensures that each worker gets a unique, but reproducible, sequence of random numbers."""
    worker_seed = SEED + worker_id;
    np.random.seed(worker_seed);
    random.seed(worker_seed);
    torch.manual_seed(worker_seed);

# Create a Generator object to manage PyTorch's internal randomness in DataLoaders.
g = torch.Generator();
# Set the seed for the generator to ensure reproducibility of DataLoader shuffling and transformations.
g.manual_seed(SEED);

Global seed set to 42 — main process is now deterministic.


In [ ]:
script_dir = os.getcwd()
folder_path = os.path.join(script_dir, 'drive', 'MyDrive', 'NEU-DET')

train_dir = os.path.join(folder_path, 'train')
train_img_dir = os.path.join(train_dir, 'images')

val_dir = os.path.join(folder_path, 'validation')
val_img_dir = os.path.join(val_dir, 'images')

In [ ]:
sorted(os.listdir(val_img_dir))

['crazing',
 'inclusion',
 'patches',
 'pitted_surface',
 'rolled-in_scale',
 'scratches']

In [ ]:
class NEUDataset(Dataset):
  def __init__(self, img_dir, transform=None):
    self.img_dir = img_dir
    self.transform = transform
    # Get the class names
    self.classes = sorted(os.listdir(img_dir))
    # Create a mapping: {'crazing': 0, 'inclusion': 1, ......}
    self.class_to_idx = {name: i for i, name in enumerate(self.classes)}

    # Create a list of every single image path and its label
    self.images = []
    for class_name in self.classes:
      class_dir = os.path.join(img_dir, class_name)
      for img_name in os.listdir(class_dir):
        # Store the path to image & its numerical label
        self.images.append((os.path.join(class_dir, img_name), self.class_to_idx[class_name]))

  def __len__(self):
    return len(self.images)

  def __getitem__(self, idx):
    # 1. Look up the address and label in our master list
    img_path, label = self.images[idx]

    # 2. Open the img file
    # We use .convert('RGB') to ensure img have 3 channels (Some may be grayscale but models usually expect 3 channels)
    image = Image.open(img_path).convert('RGB')

    if self.transform:
      image = self.transform(image)

    return image, label

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5042, 0.5042, 0.5042], std=[0.2058, 0.2058, 0.2058])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5042, 0.5042, 0.5042], std=[0.2058, 0.2058, 0.2058])
])

train_data = NEUDataset(img_dir=train_img_dir, transform=train_transforms)
val_data = NEUDataset(img_dir=val_img_dir, transform=val_transforms)

train_loader = DataLoader(
    train_data,
    batch_size=32,
    shuffle=True,
    num_workers=2,
    worker_init_fn=worker_init_fn,
    generator=g
)
val_loader = DataLoader(
    val_data,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    worker_init_fn=worker_init_fn,
    generator=g
)

Model building

In [ ]:
class CNN(nn.Module):
  def __init__(self, out_channel_1=16, out_channel_2=32, out_channel_3=64, out_channel_4=128):
    super().__init__()
    # Conv layers
    self.conv_layers = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=out_channel_1, kernel_size=3, stride=1, padding='same'),
        nn.BatchNorm2d(out_channel_1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),

        nn.Conv2d(in_channels=out_channel_1, out_channels=out_channel_2, kernel_size=3, stride=1, padding='same'),
        nn.BatchNorm2d(out_channel_2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),

        nn.Conv2d(in_channels=out_channel_2, out_channels=out_channel_3, kernel_size=3, stride=1, padding='same'),
        nn.BatchNorm2d(out_channel_3),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),

        nn.Conv2d(in_channels=out_channel_3, out_channels=out_channel_4, kernel_size=3, stride=1, padding='same'),
        nn.BatchNorm2d(out_channel_4),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
    )

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(out_channel_4*14*14, 128),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(128, 6)
    )

    # Apply Kaiming Initialization
    self._initialize_weights()

  def _initialize_weights(self):
    for m in self.modules():
        # Apply Kaiming to all Convolutional layers
        if isinstance(m, nn.Conv2d):
            # 'fan_out' preserves the magnitude in the backward pass
            nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

        # BatchNorm layers start with weight 1 and bias 0
        elif isinstance(m, nn.BatchNorm2d):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)

        # Linear layers use small random weights
        elif isinstance(m, nn.Linear):
            nn.init.normal_(m.weight, 0, 0.01)
            nn.init.constant_(m.bias, 0)

  def forward(self, x):
    x = self.conv_layers(x)
    x = self.classifier(x)
    return x

In [ ]:
model = CNN()

The Training Configuration

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
train_loss_history = []
train_acc_history = []
val_loss_history = []
val_acc_history = []

In [ ]:
def train(dataloader, model, criterion, optimizer):
  dataset_size = len(dataloader.dataset)
  model.train()
  train_loss, train_correct = 0, 0

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    yhat = model(X)
    loss = criterion(yhat, y)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    train_loss += loss.item() * len(X)
    # argmax(1) finds the index of the highest score (the predicted class)
    train_correct += (yhat.argmax(1) == y).type(torch.float).sum().item()

  avg_train_loss = train_loss / dataset_size
  avg_train_acc = train_correct / dataset_size

  train_loss_history.append(avg_train_loss)
  train_acc_history.append(avg_train_acc)

  print(f"Train Error: Accuracy: {100*avg_train_acc:>0.1f}%, Avg Loss: {avg_train_loss:>7f}")

In [ ]:
def test(dataloader, model, criterion):
  dataset_size = len(dataloader.dataset)
  model.eval()
  test_loss, test_correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      yhat = model(X)
      loss = criterion(yhat, y)

      test_loss += loss.item() * len(X)
      test_correct += (yhat.argmax(1) == y).type(torch.float).sum().item()

  avg_test_loss = test_loss / dataset_size
  avg_test_acc = test_correct / dataset_size

  val_loss_history.append(avg_test_loss)
  val_acc_history.append(avg_test_acc)

  print(f"Test Error:  Accuracy: {100*avg_test_acc:>0.1f}%, Avg Loss: {avg_test_loss:>7f}\n")
  return avg_test_loss

In [ ]:
epochs = 30
best_val_loss = float('inf')

model.to(device)
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}\n-------------------------------")
    train(train_loader, model, criterion, optimizer)
    val_loss = test(val_loader, model, criterion)

    scheduler.step(val_loss)

    if val_loss < best_val_loss:
        best_val_loss = val_loss

        torch.save(model.state_dict(), 'best_neu_model.pth')
        print("Saved best model!")

print("Done!")

Epoch 1/30
-------------------------------
Train Error: Accuracy: 63.6%, Avg Loss: 0.945481
Test Error:  Accuracy: 77.5%, Avg Loss: 0.647818

Saved best model!
Epoch 2/30
-------------------------------
Train Error: Accuracy: 82.5%, Avg Loss: 0.511130
Test Error:  Accuracy: 80.3%, Avg Loss: 0.489778

Saved best model!
Epoch 3/30
-------------------------------
Train Error: Accuracy: 89.0%, Avg Loss: 0.361864
Test Error:  Accuracy: 83.1%, Avg Loss: 0.411821

Saved best model!
Epoch 4/30
-------------------------------
Train Error: Accuracy: 88.7%, Avg Loss: 0.355513
Test Error:  Accuracy: 39.7%, Avg Loss: 2.947612

Epoch 5/30
-------------------------------
Train Error: Accuracy: 88.3%, Avg Loss: 0.402859
Test Error:  Accuracy: 88.9%, Avg Loss: 0.398000

Saved best model!
Epoch 6/30
-------------------------------
Train Error: Accuracy: 88.9%, Avg Loss: 0.327904
Test Error:  Accuracy: 75.3%, Avg Loss: 0.667418

Epoch 7/30
-------------------------------
Train Error: Accuracy: 90.1%, Avg